# **Laboratorio 1: Búsqueda en Espacios de Estados**
## **Inteligencia Artificial - DISI 2020**
### Alan Boglioli
#### Legajo: 38507

En esta práctica aplicaremos los algoritmos de búsqueda vistos en clase, viendo cómo se comportan en algunos problemas como el 8 puzzle y las Torres de Hanoi. 

El código que se usa en esta práctica está basado principalmente en el código Python que se proporciona con el libro "Artificial Intelligence: A Modern Approach" de S. Russell y P. Norvig (https://code.google.com/archive/p/aima-python/source/default/source, módulo search.py). 

**Nota:** Las _modificaciones al código_ y la _traducción_ fueron realizadas por _José Luis Ruiz Reina_ (Depto. de Ciencias de la Computación e Inteligencia Artificial de la Universidad de Sevilla).

## Pre-requisitos
* Tener una cuenta de Google (gmail)
* Tener instalado el navegador Google Chrome
* Contar con conectividad a Internet
* Conocimientos básicos de programación con el lenguaje _python_


## Entrega y uso del laboratorio

### Uso

* Antes de cualquier cosa, **cree una copia de este cuaderno:** 
  * `Click` en **File**
  * luego **Save a Copy in Drive**
  * Renombre el archivo con el siguiente formato: `APELLIDO_NOMBRE_L1.ipynb` 
* Use el _notebook_, complete las actividades y consignas que se requieran 
* Este laboratorio es una actividad *individual* pero se fomenta el intercambio de opiniones en clase

### Entrega

Una vez finalizado el laboratorio, complete [el formulario de entrega](https://docs.google.com/forms/d/e/1FAIpQLSelaLy2lvxIC7qdyKVoHLj7CZ_MpJ-5K8YDKeHwjotUqIeVHA/viewform) indicando:
 * `Apellido`
 * `Nombre`
 * `Nro Legajo`
 * `Turno (tarde o noche)`
 * `link de su notebook`. El mismo se obtiene si realiza click en **Compartir/Share** (esquina superior derecha) y luego en **Obtener link para compartir/Get shareable link**  
 
NOTA: No se aceptarán otras formas de entrega distintas a la mencionada.

**Fecha límite de entrega:** se indicará en clase o en el aula virtual. 

# Consideraciones

En el ejercicio final se ha cambiado la **BúsquedaEnProfundidad** por una **BúsquedaEnProfundidadAcotada** para asegurar que la búsqueda sea completa, es decir, termine.

# PARTE I: Representación de Espacios de Estados

Recuérde que según lo que se ha visto en clase, la implementación de la 
representación de un problema de espacio de estados consiste en:

 * Representar **estados** y **acciones** mediante una ***estructura de datos***.
 * Definir **estado inicial**, **estado final**, **acciones**, **función sucesor** y **coste de aplicar accion**, si el problema tiene coste.


La siguiente  clase `Problema` representa el esquema general de cualquier problema de espacio de estados. 

Un problema concreto será una *subclase* de `Problema`, y requerirá implementar o sobreescribir las funciones: `acciones`, `aplica` y eventualmente `init`, `es_estado_final` y `coste_de_aplicar_accion`.


In [0]:
class Problema(object):
    """

    Clase abstracta para un problema de espacio de estados. 
    
    Los problemas concretos habría que definirlos como subclases de Problema, implementando
    acciones, aplica y eventualmente __init__, es_estado_final y
    coste_de_aplicar_accion. 
    
    Una vez hecho esto, se han de crear instancias de
    dicha subclase, que serán la entrada a los distintos algoritmos de
    resolución mediante búsqueda.

    """  
    
    def __init__(self, estado_inicial, estado_final=None):
        """
        El constructor de la clase especifica el estado inicial y puede que un estado_final, si es que es único. 
        Las subclases podrían añadir otros argumentos
        """  
        self.estado_inicial = estado_inicial
        self.estado_final = estado_final

    def acciones_aplicables(self, estado):
        """
        Devuelve las acciones aplicables a un estado dado. 
        Lo normal es que aquí se devuelva una lista, pero si hay muchas se podría devolver
        un iterador, ya que sería más eficiente.
        """
        pass

    def aplica(self, estado, accion):
        """ 
        Devuelve el estado resultante de aplicar accion a estado. 
        Se supone que accion es aplicable a estado (es decir, debe ser una de las acciones de self.acciones(estado)).
        """
        pass

    def es_estado_final(self, estado):
        """
        Devuelve True cuando estado es final. 
        Por defecto, compara con el estado final, si éste se hubiera especificado al constructor. 
        Si se da el caso de que no hubiera un único estado final, o se definiera mediante otro tipo de comprobación, habría que redefinir este método
        """ 
        return estado == self.estado_final

    def coste_de_aplicar_accion(self, estado, accion):
        """
        Devuelve el coste de aplicar accion a estado. 
        Por defecto, este
        coste es 1. 
        Reimplementar si el problema define otro coste """ 
        return 1

## Un ejemplo: el **problema de las jarras**

**Enunciado**:
  - Se tienen dos jarras, de 4 y 3 litros respectivamente.
  - Ninguna de ellas tiene marcas de medición.
  - Se tiene una bomba que permite llenar las jarras de agua.
  - Averiguar cómo se puede lograr tener exactamente 2 litros de agua en la jarra de 4 litros de capacidad.

**Estado inicial**: el estado inical se representa con la tupla (0 0). 




### ¿Cual sería la representación para un estado final?




In [0]:
un_estado_final_jarras = (2, 2)
un_estado_final_jarras

(2, 2)


**Acciones**:
  - Llenar la jarra de 4 litros con la bomba.
  - Llenar la jarra de 3 litros con la bomba.
  - Vaciar la jarra de 4 litros en el suelo.
  - Vaciar la jarra de 3 litros en el suelo.
  - Trasvasar de la jarra de 4 litros a la jarra de 3 litros. Si se llena la jarra de 3 litros, el sobrante permanece en la jarra de 4 litros.
  - Trasvasar de jarra de 3 litros a la jarra de 4 litros. Si se llena la jarra de 4 litros, el sobrante permanece en la jarra de 3 litros.


La clase `Jarras` implementada a continuación hereda de `Problema` y formaliza este problema implementando las funciones acorde.

In [0]:
class Jarras(Problema):
    """
    Problema de las jarras:
      Representaremos los estados como tuplas (x,y) de dos números enteros, donde 
        x es el número de litros de la jarra de 4 
        y es el número de litros de la jarra de 3
    """
    
    def __init__(self):
        self.estado_inicial = (0,0)

    def acciones_aplicables(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def aplica(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        elif accion =="trasvasar de jarra de 3 a jarra de 4":
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def es_estado_final(self,estado):
        return estado[0]==2       

 ---
## **Ejercicio 1**
 
 **Completar el código** que se presenta a continuación, en los lugares marcados con interrogantes para entender el funcionamiento de la implementación del problema de las jarras de la celda anterior y responder las preguntas que se presenan.

**NO modifique el nombre de las variables**




### 1. ¿Cuales son las acciones disponibles a partir del estado inicial en el problema de las jarras?


In [0]:
# instanciar el problema
problema_jarras_4_3 = Jarras()

# obtener el estado inicial
estado_inicial_jarras_4_3 =  problema_jarras_4_3.estado_inicial
print('El estado incial es {}'.format(estado_inicial_jarras_4_3))

#obtener las acciones
acciones_jarras_4_3_inicial = problema_jarras_4_3.acciones_aplicables(estado_inicial_jarras_4_3)
print('Acciones disponibles en estado {} --> {}'.format(estado_inicial_jarras_4_3,acciones_jarras_4_3_inicial))


El estado incial es (0, 0)
Acciones disponibles en estado (0, 0) --> ['llenar jarra de 4', 'llenar jarra de 3']


### 2. ¿Cuál es el coste de aplicar la segunda acción disponible al estado inicial?

In [0]:
# obtener el coste de la accion indicada
coste_estado_1_jarras_4_3 = problema_jarras_4_3.coste_de_aplicar_accion(estado_inicial_jarras_4_3, acciones_jarras_4_3_inicial[1])
print('El coste de esa acción es {}'.format(coste_estado_1_jarras_4_3))

El coste de esa acción es 1


### 3. ¿Es el estado resultado un estado el final? 

In [0]:
# obtener el estado resultado
estado_1_jarras_4_3 = problema_jarras_4_3.aplica(estado_inicial_jarras_4_3, acciones_jarras_4_3_inicial[1])
print('Estado obtenido luego de aplicar accion {} --> {}'.format(acciones_jarras_4_3_inicial[1],estado_1_jarras_4_3))

# checkear si es estado final del problema
print('Es este el estado final ? --> {}'.format(problema_jarras_4_3.es_estado_final(estado_1_jarras_4_3)))


Estado obtenido luego de aplicar accion llenar jarra de 3 --> (0, 3)
Es este el estado final ? --> False


---
## **Ejercicio 2**

Clase que implementa un problema de espacio de estados para la representación del problema del **8-puzzle**. 


In [0]:
class Ocho_Puzzle(Problema):
    """
    Los estados serán tuplas de nueve elementos, permutaciones de los números del 0 al 8 (el 0 es el hueco). 
    Representan la disposición de las fichas en el tablero, leídas por filas de arriba a abajo, y dentro de cada fila, de izquierda a derecha. 
    
    Por ejemplo, el estado final será la tupla (1, 2, 3, 8, 0, 4, 7, 6, 5) y representa el estado del mundo real dado por

    | 1 | 2 | 3 |
    | 8 | 0 | 4 |
    | 7 | 6 | 5 |
    
    Las cuatro acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y "Mover hueco derecha", respectivamente.
    """

    def __init__(self,tablero_inicial):
        self.estado_inicial = tablero_inicial 
        self.estado_final = (1, 2, 3, 8, 0, 4, 7, 6, 5) 

    def acciones_aplicables(self,estado):
        pos_hueco=estado.index(0)
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (0,3,6): 
          accs.append("Mover hueco izquierda")
        if pos_hueco not in (6,7,8): 
          accs.append("Mover hueco abajo")
        if pos_hueco not in (2,5,8):
          accs.append("Mover hueco derecha") 
        return accs     

    def aplica(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        return tuple(l)    

 **Completar el código** que se presenta a continuación, en los lugares marcados con interrogantes para validar el funcionamiento de la implementación del problema del 8 puzzle y responder las preguntas a continuación.
 
 **NO modifique el nombre de las variables**




   ### 1. Cuáles son las acciones disponibles a partir del siguiente estado inicial ?
   
```
                                  | 2 | 8 | 3 |
                                  | 1 | 6 | 4 |
                                  | 7 | 0 | 5 |
```
   

In [0]:
# instanciar un problema de 8 puzzle usando la clase anterior
p8p_1 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))

# obtener el estado inicial
estado_0 = p8p_1.estado_inicial

# obtener las acciones aplicables de ese estado
acciones_e0 = p8p_1.acciones_aplicables(estado_0)

print('Las acciones disponibles desde el estado inicial {} son: \n {}'.format(estado_0, acciones_e0))

Las acciones disponibles desde el estado inicial (2, 8, 3, 1, 6, 4, 7, 0, 5) son: 
 ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']


### 2. Cual es el estado final ?


In [0]:
# obtener el estado final desde el problema
estado_n = p8p_1.estado_final

print('El estado final es {}'.format(estado_n))

El estado final es (1, 2, 3, 8, 0, 4, 7, 6, 5)


### 3. Cual es el estado que se obtiene si aplico la tercera acción disponible al estado inicial ?




In [0]:
# obtener la tercera accion disponible desde el estado incial
tercera_accion = acciones_e0[2]
# aplicarla al estado incial
estado_a3 = p8p_1.aplica(estado_0, tercera_accion)

print('El resultado de aplicar la tercera accion disponible (\"{}\") en el estado inicial {} es {}'.format(tercera_accion,estado_0,estado_a3))

El resultado de aplicar la tercera accion disponible ("Mover hueco derecha") en el estado inicial (2, 8, 3, 1, 6, 4, 7, 0, 5) es (2, 8, 3, 1, 6, 4, 7, 5, 0)


### 4. Valide si es el estado final


In [0]:
# usar el problema para saber si es estado inicial
es_final = p8p_1.es_estado_final(estado_a3)

print('Es {} el estado final ? --> {}'.format(estado_a3, es_final))

Es (2, 8, 3, 1, 6, 4, 7, 5, 0) el estado final ? --> False


---
## **Ejercicio 3**

A continuación tenemos algunas clases que nos permitirán hacer una implementación Genérica del **Rompecabezas de las Torres de Hanoi**

El rompecabezas de las Torres de Hanoi consta de tres varillas verticales y un número de discos, que determinará la complejidad del problema, todos de distinto tamaño y apilados de mayor a menor radio en la primera varilla.

El objetivo del juego es pasar todos los discos de la primera a la última varilla, siguiendo tres simples reglas:
1. Se desplaza un disco cada vez.
2. Solo se pueden desplazar los discos de arriba de las varillas.
3. No se puede colocar un disco sobre otro más pequeño.

En esta parte de la práctica se mostrará cómo implementar el rompecabezas de las Torres de Hanoi como un problema de espacio de estados y se aplicarán distintos algoritmos de búsqueda para resolverlo.

Para implementar un problema de espacio de estados copiamos aquí las clases de objetos proporcionadas por el módulo `problema_espacio_estados` (`problema_espacio_estados.py`).

El código de este módulo incluye la definición de dos clases genéricas para la formulación de un problema de busqueda. Separamos la clase original `Problema` en dos, de manera que las acciones se representan al heredar de la clase `Acción` y el problema con su estado inicial, final y función sucesor a partir de la clase `ProblemaEspacioEstados`

In [0]:

class Acción:

    def __init__(self, nombre='', aplicabilidad=None, aplicación=None,
                 coste=None):
        self.nombre = nombre
        self.aplicabilidad = aplicabilidad
        self.aplicación = aplicación
        self.coste = coste

    def es_aplicable(self, estado):
        if self.aplicabilidad is None:
            raise NotImplementedError('Aplicabilidad de la acción no implementada')
        else:  return self.aplicabilidad(estado)

    def aplicar(self, estado):
        if self.aplicar is None:
            raise NotImplementedError('Aplicación de la acción no implementada')
        else: return self.aplicación(estado)

    def coste_de_aplicar(self, estado):
        if self.coste is None:  return 1
        else:  return self.coste(estado)
        
    def __str__(self):
        return 'Acción: {}'.format(self.nombre)

In [0]:
class ProblemaEspacioEstados(Problema):

    def __init__(self, acciones, estado_inicial=None, estados_finales=None):
        if not isinstance(acciones, list):
            raise TypeError('Debe proporcionarse una lista de acciones')
        self.acciones = acciones
        self.estado_inicial = estado_inicial
        self.estados_finales = estados_finales

    def es_estado_final(self, estado):
        return estado in self.estados_finales
        
    def acciones_aplicables(self, estado):
        return (acción for acción in self.acciones if acción.es_aplicable(estado))

**El primer paso es decidir cómo se van a implementar los estados**

Para el rompecabezas de las Torres de Hanoi una opción es hacerlo mediante una lista que guarde para cada varilla el conjunto de los discos que hay en ella.

### Implemente dos estados para el problema de las torres con 2 discos donde :
 * `estado1` : la varilla 1 tiene el disco 2 y la varilla 3 el disco 1
 * `estado2` : la varilla 1 tiene el disco 1 y la varilla 3 el disco 2
 

In [0]:
estado1 = [{2}, set(), {1}]
estado2 = [{1}, set(), {2}]

A continuación **hay que implementar las acciones como instancias de la clase** `Acción`, proporcionando un `nombre`, una `función de aplicabilidad` y una `función de aplicación para cada acción`. 

Por ejemplo, la acción `De 1 a 3` que mueve un disco de la primera a la tercera varilla se puede implementar de la siguiente manera:

In [0]:
def está_vacía(estado, varilla):
    return not bool(estado[varilla - 1])

def disco_superior(estado, varilla):
    return min(estado[varilla - 1])

def aplicabilidad(estado):
    return (not está_vacía(estado, 1) and
            (está_vacía(estado, 3) or
             disco_superior(estado, 1) < disco_superior(estado, 3)))
    
def quitar_disco(estado, varilla):
    disco = disco_superior(estado, varilla)
    estado[varilla - 1].remove(disco)
    return disco

def poner_disco(estado, varilla, disco):
    estado[varilla - 1].add(disco)

def aplicación(estado):
    import copy
    nuevo_estado = copy.deepcopy(estado)
    disco = quitar_disco(nuevo_estado, 1)
    poner_disco(nuevo_estado, 3, disco)
    return nuevo_estado
    
a13 = Acción('De 1 a 3', aplicabilidad, aplicación)

Normalmente las acciones se pueden **agrupar en distintos tipos**, cada uno de los cuales puede ser implementado de manera abstracta mediante una clase que herede de la clase `Acción`.

Para el rompecabezas de las Torres de Hanoi, todas las acciones son del tipo mover un disco de una varilla a otra. En este caso, consideramos que el coste de mover un disco es siempre 1, el valor por defecto. 

En caso de que fuera distinto, al crear una instancia de la clase `Acción` se puede proporcionar una función `coste`, o bien al heredar de la clase `Acción` se puede redefinir el método `coste_de_aplicar`.

A continuación se implementó una sub-clase de `Accion` que representa el movimiento de discos. 



In [0]:
import copy

class MoverDisco(Acción):
  
    def __init__(self, i, j):
        nombre = 'De {} a {}'.format(i, j)
        super().__init__(nombre)
        self.varilla_de = i 
        self.varilla_a = j    

    def está_vacía(self, estado, varilla):
        return not bool(estado[varilla - 1])  

    def disco_superior(self, estado, varilla):
        try :
            return min(estado[varilla - 1])
        except :
            return float('inf')

    def es_aplicable(self, estado):
        varilla_de_llena = not self.está_vacía(estado, self.varilla_de) 
        varilla_a_vacia = self.está_vacía(estado, self.varilla_a)
        if varilla_de_llena and varilla_a_vacia:
            return True 
        elif not varilla_a_vacia:
            disco_superior_de_menor_disco_superior_a = self.disco_superior(estado, self.varilla_de) < self.disco_superior(estado, self.varilla_a)   
            return disco_superior_de_menor_disco_superior_a
        
    def quitar_disco(self, estado, varilla):
        disco = self.disco_superior(estado, varilla)
        estado[varilla - 1].remove(disco) 
        return disco

    def poner_disco(self, estado, varilla, disco):
        estado[varilla - 1].add(disco)    

    def aplicar(self, estado):
        nuevo_estado = copy.deepcopy(estado) 
        disco = self.quitar_disco(nuevo_estado, self.varilla_de) 
        self.poner_disco(nuevo_estado, self.varilla_a, disco)
        return nuevo_estado

Finalmente, **un problema de espacio de estados se implementa como una instancia de la clase `ProblemaEspacioEstados`**, proporcionando una lista de `acciones`, un `estado inicial` y una lista de `estados finales`.

**Completar el código** que se presenta a continuación, en los lugares marcados con interrogantes para instanciar un problema de espacio de estados que permita resolver las _torres de hanoi_ con dos discos y responder las preguntas.

**NO cambiar el nombre de las variables**

El estado inicial para el problema con dos discos sería el siguiente : 

```  

                    |  |  |
                    1  |  | 
                    2  |  | 
                  -----------   

```

In [0]:
# crea una lista de acciones Mover Disco
acciones =  [MoverDisco(i, j) for i in range(1, 4) for j in range(1, 4) if i != j]
for a in acciones:
  print(a.nombre)
# instanciar estado incial como lista de tuplas
estado_inicial = [{1,2}, set(), set()] # acciones[0]
# instancia el estado final como lista de tuplas
estado_final = [set(), set(), {1, 2}]
# instanciar el ProblemaEspacioEstados
Torres_Hanoi_2_discos = ProblemaEspacioEstados(acciones, estado_inicial, [estado_final]) # Considera múltiples estados finales

De 1 a 2
De 1 a 3
De 2 a 1
De 2 a 3
De 3 a 1
De 3 a 2


### 1. El `estado1` definido anteriormente, es estado final?


In [0]:
# usar el problema Torres_Hanoi_2_discos para saber si estado1 es final
es_estado1_final = Torres_Hanoi_2_discos.es_estado_final(estado1) 
print('Es el estado {} un estado final? --> {}'.format(estado1,es_estado1_final))

Es el estado [{2}, set(), {1}] un estado final? --> False


### 2. El estado resultado de aplicar la acción `a13` sobre el `estado2`, es final de acuerdo a este problema?
  

In [0]:
# usar a13 sobre estado2
estado3 = a13.aplicar(estado2)
# usar Torres_Hanoi_2_discos para ver si estado3 es final
es_estado3_final =  Torres_Hanoi_2_discos.es_estado_final(estado3)
print('El resultado de aplicar {} al estado {} es {}, es un estado final? --> {}'.format(a13,estado2,estado3,es_estado3_final))

El resultado de aplicar Acción: De 1 a 3 al estado [{1}, set(), {2}] es [set(), set(), {1, 2}], es un estado final? --> True


### 3. Cuales son las acciones aplicables desde el `estado1`?
  

In [0]:
print('Las acciones aplicables en el estado {} son: \n'.format(estado1))
for acción in Torres_Hanoi_2_discos.acciones_aplicables(estado1): # usar  Torres_Hanoi_2_discos para tener las acciones aplicables en estado1
    print(acción.nombre)

Las acciones aplicables en el estado [{2}, set(), {1}] son: 

De 1 a 2
De 3 a 1
De 3 a 2


# PARTE II. Experimentando

El procedimiento para realizar una búsqueda en un espacio de estados consiste en crear una instancia de una clase que implemente un algoritmo de búsqueda, proporcionando los argumentos necesarios, y aplicar el método buscar de esa instancia al problema de espacio de estados.

Las clases correspondientes a los algoritmos de búsqueda más comunes son las siguientes:
* `BúsquedaEnAnchura`
* `BúsquedaEnProfundidad`
* `BúsquedaPrimeroElMejor`: hay que proporcionar la función de evaluación heurística `f`.
* `BúsquedaÓptima`
* `BúsquedaAEstrella`: hay que proporcionar la función de estimación del coste `h`.

Adicionalmente, todas las clases anteriores admiten establecer el argumento `detallado` a `True`, para que al realizar una búsqueda se imprima por pantalla su traza.

---
## **Ejercicio 4**


A cotinuación se presentan una serie de clases accesorias que nos van a permitir realizar las búsquedas que necesitamos.

In [0]:
import collections
import heapq
import types


class ListaNodos(collections.deque):

    def añadir(self, nodo): 
      self.append(nodo)
    
    def vaciar(self): 
      self.clear()
    
    def __contains__(self, nodo):
        return any(x.estado == nodo.estado for x in self)

class PilaNodos(ListaNodos):
    
    def sacar(self):  
      return self.pop()

class ColaNodos(ListaNodos):

    def sacar(self):  
      return self.popleft()

class ColaNodosConPrioridad:
    
    def __init__(self):
        self.nodos = []
        self.nodo_generado = 0
    
    def añadir(self, nodo):
        heapq.heappush(self.nodos, (nodo.heurística, self.nodo_generado, nodo))
        self.nodo_generado += 1
    
    def sacar(self):
        return heapq.heappop(self.nodos)[2]
    
    def vaciar(self):
        self.__init__()
    
    def __iter__(self):
        return iter(self.nodos)
    
    def __contains__(self, nodo):
        return any(x[2].estado == nodo.estado and x[2].heurística <= nodo.heurística for x in self.nodos)


class NodoSimple:
    
    def __init__(self, estado, padre=None, acción=None):
        self.estado = estado # Estado que representa el nodo
        self.padre = padre # Nodo padre
        self.acción = acción # Acción aplicada por la que se llegó al nodo
    
    def es_raíz(self): 
      return self.padre is None # El nodo raíz es el único sin padre
    
    def sucesor(self, acción):
        Nodo = self.__class__
        return Nodo(acción.aplicar(self.estado), self, acción) # Nueva instancia de nodo con estado a partir de aplicar la acción al estado actual
    
    def solución(self):
        if self.es_raíz():  acciones = [] # Caso base. No se aplican acciones para llegar a la razón, es el padre.
        else:
            acciones = self.padre.solución() # Camino solución del padre
            acciones.append(self.acción.nombre) # Camino solución del padre + acción para llegar a nodo actual
        return acciones
    
    def __str__(self):  return 'Estado: {}'.format(self.estado)

class NodoConProfundidad(NodoSimple):
    
    def __init__(self, estado, padre=None, acción=None):
        super().__init__(estado, padre, acción)
        if self.es_raíz():  
            self.profundidad = 0 # El nodo raíz tiene profundidad 0
        else:   
            self.profundidad = padre.profundidad + 1 # Aumenta en 1 la profundidad del padre: nuevo nivel de profundidad.
    
    def __str__(self):
        return 'Estado: {0}; Prof: {1}'.format(self.estado, self.profundidad)

class NodoConHeurística(NodoSimple):
    
    def __init__(self, estado, padre=None, acción=None):
        super().__init__(estado, padre, acción)
        if self.es_raíz():
            self.profundidad = 0
            self.coste = 0
        else:
            self.profundidad = padre.profundidad + 1
            self.coste = padre.coste + acción.coste_de_aplicar(padre.estado)
        self.heurística = self.f(self)
    
    @staticmethod
    def f(nodo):
        return 0
    
    def __str__(self):
        return 'Estado: {0}; Prof: {1}; Heur: {2}; Coste: {3}'.format(
            self.estado, self.profundidad, self.heurística, self.coste)



In [0]:

class BúsquedaGeneral:

    def __init__(self, detallado=False):
        self.detallado = detallado
        if self.detallado:  
            self.Nodo = NodoConProfundidad
        else:  
            self.Nodo = NodoSimple
        self.explorados = ListaNodos()
    
    def es_expandible(self, nodo): 
        return True
    
    def expandir_nodo(self, nodo, problema):
        return [nodo.sucesor(acción) for acción in problema.acciones_aplicables(nodo.estado)]
    
    def es_nuevo(self, nodo):
        return (nodo not in self.frontera and
                nodo not in self.explorados)
    
    def buscar(self, problema):
        self.frontera.vaciar()
        self.explorados.vaciar()
        self.frontera.añadir(self.Nodo(problema.estado_inicial))
    
        while True:
          if not self.frontera:
              return None
          nodo = self.frontera.sacar() # Sacar
          if self.detallado:
              print('{0}Nodo: {1}'.format('  ' * nodo.profundidad, nodo))
          
          if problema.es_estado_final(nodo.estado): # Evaluar
              return nodo.solución()
          
          self.explorados.añadir(nodo)
          
          if self.es_expandible(nodo):
              nodos_hijos = self.expandir_nodo(nodo, problema) # Generar
              for nodo_hijo in nodos_hijos:
                  if self.es_nuevo(nodo_hijo): # Evita repeticiones
                      self.frontera.añadir(nodo_hijo) # Poner


class BúsquedaEnAnchura(BúsquedaGeneral):
    
    def __init__(self, detallado=False):
        super().__init__(detallado)
        self.frontera = ColaNodos()

class BúsquedaEnProfundidad(BúsquedaGeneral):

    def __init__(self, detallado=False):
        super().__init__(detallado)
        self.frontera = PilaNodos()
        self.explorados = PilaNodos()

    def añadir_vaciando_rama(self, nodo):
        if self:
            while True:
                último_nodo = self.pop()
                if último_nodo == nodo.padre:
                    self.append(último_nodo)
                    break
        self.append(nodo)
        self.explorados.añadir = types.MethodType(añadir_vaciando_rama,self.explorados)

class BúsquedaEnProfundidadAcotada(BúsquedaEnProfundidad):
    def __init__(self, cota, detallado=False):
        super().__init__(detallado)
        self.Nodo = NodoConProfundidad
        self.cota = cota

    def es_expandible(self, nodo):
        return nodo.profundidad < self.cota

class BúsquedaEnProfundidadIterativa:
    def __init__(self, cota_final, cota_inicial=0, detallado=False):
        self.cota_inicial = cota_inicial
        self.cota_final = cota_final
        self.detallado = detallado

    def buscar(self, problema):
        for cota in range(self.cota_inicial, self.cota_final):
            bpa = BúsquedaEnProfundidadAcotada(cota, self.detallado)
            solución = bpa.buscar(problema)
            if solución:
                return solución

class BúsquedaPrimeroElMejor(BúsquedaGeneral):
    def __init__(self, f, detallado=False):
        super().__init__(detallado)
        self.Nodo = NodoConHeurística
        self.Nodo.f = staticmethod(f)
        self.frontera = ColaNodosConPrioridad()
        self.explorados = ListaNodos()
        self.explorados.__contains__ = types.MethodType(
            lambda self, nodo: any(x.estado == nodo.estado and
                                   x.heurística <= nodo.heurística
                                   for x in self),
            self.explorados)
        
class BúsquedaÓptima(BúsquedaPrimeroElMejor):
    def __init__(self, detallado=False):
        def coste(nodo):  return nodo.coste
        super().__init__(coste, detallado)

class BúsquedaAEstrella(BúsquedaPrimeroElMejor):
    def __init__(self, h, detallado=False):
        def coste(nodo):
            return nodo.coste
        def f(nodo):
            return coste(nodo) + h(nodo)
        super().__init__(f, detallado)

#### 1. Aplicar la búsqueda en anchura al problema de las torres de Hanoi de 2 discos anteriormente implementado. Usa la búsqueda detallada para entender como funciona. 

In [0]:
# instanciar una BusquedaEnAnchura detallada
b_anchura = BúsquedaEnAnchura(True) 
solucion_anchura_hanoi_2 = b_anchura.buscar(Torres_Hanoi_2_discos)
print(solucion_anchura_hanoi_2)

Nodo: Estado: [{1, 2}, set(), set()]; Prof: 0
  Nodo: Estado: [{2}, {1}, set()]; Prof: 1
  Nodo: Estado: [{2}, set(), {1}]; Prof: 1
    Nodo: Estado: [set(), {1}, {2}]; Prof: 2
    Nodo: Estado: [set(), {2}, {1}]; Prof: 2
      Nodo: Estado: [{1}, set(), {2}]; Prof: 3
      Nodo: Estado: [set(), set(), {1, 2}]; Prof: 3
['De 1 a 2', 'De 1 a 3', 'De 2 a 3']


#### 2. Aplicar la búsqueda en profundidad al problema de las torres de Hanoi de 2 discos anteriormente implementado. Use la búsqueda detallada para entender como funciona

In [0]:
# instanciar una BusquedaEnProfundidad detallada
b_profundidad = BúsquedaEnProfundidad(True)
solucion_profundidad_hanoi_2 = b_profundidad.buscar(Torres_Hanoi_2_discos)
print(solucion_profundidad_hanoi_2)

Nodo: Estado: [{1, 2}, set(), set()]; Prof: 0
  Nodo: Estado: [{2}, set(), {1}]; Prof: 1
    Nodo: Estado: [set(), {2}, {1}]; Prof: 2
      Nodo: Estado: [set(), {1, 2}, set()]; Prof: 3
      Nodo: Estado: [{1}, {2}, set()]; Prof: 3
        Nodo: Estado: [{1}, set(), {2}]; Prof: 4
          Nodo: Estado: [set(), set(), {1, 2}]; Prof: 5
['De 1 a 3', 'De 1 a 2', 'De 3 a 1', 'De 2 a 3', 'De 1 a 3']


Podemos parametrizar la implementación del rompecabezas de las Torres de Hanoi para que dependa del número `n` de discos. Para ello basta implementar una clase que herede de la clase `ProblemaEspacioEstados`. Aprovechamos también para, en lugar de enumerar los estados finales, realizar una descripción declarativa de los mismos redefiniendo el método `es_estado_final`.

In [0]:
class TorresHanoi(ProblemaEspacioEstados):
    def __init__(self, n):
        acciones = [MoverDisco(i, j) for i in range(1, 4) for j in range(1, 4) if i != j]
        estado_inicial = [set(range(1, n + 1)), set(), set()]
        super().__init__(acciones, estado_inicial)
        self.n = n
    def es_estado_final(self, estado):
        return estado[2] == set(range(1, self.n + 1))

#### 3. Cuál es el tiempo que le toma a las búsquedas en anchura y optima ejecutar la búsqueda para el problema de las Torres de Hanoi con 8 discos ?


In [0]:
# Prevenir excepciones por exceso de stack al realizar llamadas recursivas
import sys
sys.setrecursionlimit(10000)

import time
# Instanciar un problema TorresHanoi con 8 discos
Torres_Hanoi_8_discos = TorresHanoi(8)
# Instanciar una BusquedaEnAnchura NO detallada
b_anchura = BúsquedaEnAnchura()
# Instanciar una BusquedaOptima NO detallada
b_optima = BúsquedaÓptima()

In [0]:
# inicia contador de tiempo
tic = time.time()
# usar la busqueda en anchura para buscar la solucion del problema
b_anchura.buscar(Torres_Hanoi_8_discos)
# contador de finalización
toc = time.time()
# tiempo total
tiempo_total_anchura = toc-tic
print('Tiempo total de la búsqueda en anchura fue {} segundos'.format(tiempo_total_anchura))

Tiempo total de la búsqueda en anchura fue 8.549259424209595 segundos


In [0]:
tic = time.time()
# usar la búsqueda optima para buscar la solucion del problema
b_optima.buscar(Torres_Hanoi_8_discos)
toc = time.time()
total_tiempo_optima = toc - tic
print('Tiempo total de la búsqueda optima fue {} segundos'.format(total_tiempo_optima))

Tiempo total de la búsqueda optima fue 7.69922947883606 segundos


Con un número de discos igual a 8, el coste en tiempo de los algoritmos de búsqueda en anchura y profundidad comienza a no ser asumible, por lo que debemos pasar a realizar una búsqueda informada.


Para poder aplicar la búsqueda $A^*$, es un requisito necesario definir una función que para cada nodo estime el coste de una solución óptima desde el estado de ese nodo (que en nuestra implementación está guardado en el atributo `estado` de la clase que implementa a estos últimos).

Vemos un ejemplo de una implementación de heurística

In [0]:
def h(nodo):
    estado = nodo.estado
    return len(estado[0]) + len(estado[1])

#### 4. ¿Como está definida la heurística `h` del bloque de código anterior?

In [0]:
h_hanoi_respuesta = 'Definida como la cantidad de discos mal posicionados, que no se encuentran en la última varilla'

# AGREGADO
class MockNodo:
  def __init__(self, estado): self.estado = estado
n1 = MockNodo([{1,2}, set(), set()])
n2 = MockNodo([{1}, {2}, set()])
n3 = MockNodo([{2}, {1}, set()])
n4 = MockNodo([set(), {1}, {2}])
n5 = MockNodo([{2}, set(), {1}])
n5 = MockNodo([{1,2}, {3,4}, {5,6,7,8}])
h_hanoi_respuesta, h(n1), h(n2), h(n3), h(n4), h(n5)

('Definida como la cantidad de discos mal posicionados, que no se encuentran en la última varilla',
 2,
 2,
 2,
 1,
 4)

#### 5. ¿ Cuánto tarda una búsqueda A* con la heurística anterior para el mismo problema ?

In [0]:
# Instancie una BusquedaAEstrella NO detallada
b_a_estrella = BúsquedaAEstrella(h)
tic = time.time()
b_a_estrella.buscar(Torres_Hanoi_8_discos)
toc = time.time()
tiempo_total_a_estrella = toc - tic
print('Tiempo total de la búsqueda A* fue {} segundos'.format(tiempo_total_a_estrella))

Tiempo total de la búsqueda A* fue 8.25515079498291 segundos


# PARTE III. Estadísticas

La siguientes definiciones nos van a permitir experimentar con distintos estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. 

Además se van a contar el número de nodos analizados durante la búsqueda:


In [0]:
class Problema_con_Analizados(Problema):
    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda."""          
    def __init__(self, problema):
        self.estado_inicial = problema.estado_inicial
        self.problema = problema
        self.analizados  = 0
    
    def acciones_aplicables(self, estado): return self.problema.acciones_aplicables(estado)
    
    def aplica(self, estado, accion): return self.problema.aplica(estado, accion)
    
    def es_estado_final(self, estado):
        self.analizados += 1
        return self.problema.es_estado_final(estado)
    
    def coste_de_aplicar_accion(self, estado, accion):
        return self.problema.coste_de_aplicar_accion(estado,accion)




In [0]:

class MoverHueco(Acción):
  
    def __init__(self, direccion):
        nombre = 'Mover hueco a {}'.format(direccion)
        super().__init__(nombre)
        self.direccion = direccion

    def es_aplicable(self, estado):
        pos_hueco=estado.index(0)
        if (pos_hueco not in (0,1,2)) and (self.direccion == 'arriba') :
            return True
        elif (pos_hueco not in (0,3,6)) and (self.direccion == 'izquierda'): 
            return True
        elif (pos_hueco not in (6,7,8)) and (self.direccion == 'abajo'):
            return True
        elif (pos_hueco not in (2,5,8)) and (self.direccion == 'derecha'):
            return True
        else:
            return False
        

    def aplicar(self, estado):
        pos_hueco = estado.index(0)
        l = list(estado)
        if self.direccion == "arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        if self.direccion == "abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        if self.direccion == "derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        if self.direccion == "izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        return tuple(l)    

In [0]:
class OchoPuzzleEstados(ProblemaEspacioEstados):
    def __init__(self, estado_inicial):
        acciones = [MoverHueco(direccion) for direccion in ['arriba','abajo','izquierda','derecha']]
        super().__init__(acciones, estado_inicial)
            
    def es_estado_final(self, estado):
        return estado == (1, 2, 3, 8, 0, 4, 7, 6, 5)

In [0]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho puzzle, 
        con un estado inicial dado y (cuando el algoritmo lo necesite)  una heurística dada.
       Ejemplo de uso:
       >>> resuelve_ocho_puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5),búsqueda_a_estrella,h2_ocho_puzzle)
       Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 
                  'Mover hueco abajo', 'Mover hueco derecha']
       Algoritmo: búsqueda_a_estrella
       Heurística: h2_ocho_puzzle
       Longitud de la solución: 5. Nodos analizados: 7
       """
    p8p=Problema_con_Analizados(OchoPuzzleEstados(estado_inicial))
    tic = time.time()
    
    sol = (algoritmo(h).buscar(p8p) if h else algoritmo().buscar(p8p)) 

    # En caso de aplicar cota y que no encuentre la solución
    if sol is None:
      sol = ['No se encontró']

    toc = time.time()
    tiempo = toc - tic
    print("[ {0} ]".format(algoritmo.__name__))
    print("Solución: {0}".format(sol))
    if h:  print("Heurística: {0}".format(h.__name__))
    else:
        pass
    print("Longitud de la solución: {0}. Nodos analizados: {1}, Tiempo de búsqueda: {2}".format(len(sol),p8p.analizados,tiempo))
    return {'longitud' : len(sol),'analizados' : p8p.analizados,'tiempo' : tiempo,'solucion':sol}

---
### **Ejercicio 5**

Definir las siguientes funciones heurísticas para el 8 puzzle.
* `h1_ocho_puzzle(estado)`: cuenta el número de casillas mal colocadas respecto del estado final.
* `h2_ocho_puzzle_estado(estado)`: suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.

Ejemplos de resultados esperados:

```
>>> h1_ocho_puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
4
>>> h2_ocho_puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
5
>>> h1_ocho_puzzle((5,2,3,0,4,8,7,6,1))
4
>>> h2_ocho_puzzle((5,2,3,0,4,8,7,6,1))
11
```

In [0]:
def h1_ocho_puzzle(nodo):
  estado = nodo.estado
  estado_final = (1, 2, 3, 8, 0, 4, 7, 6, 5)
  h = 8 # Comienza suponiendo que todas están mal colocadas, para restar 1 cuando se encuentre alguna bien colocada
  # Implementar suma del número de casillas mal colocadas respecto del estado final
  i = 0
  for c in estado_final:
    if c == 0: # Ignorar hueco
      i += 1
      continue

    if c == estado[i]:
      h -= 1

    i += 1

  return h

# Aserciones de resultados esperados
class MockNode:
  def __init__(self, estado): self.estado = estado
h1 = h1_ocho_puzzle(MockNode((2, 8, 3, 1, 6, 4, 7, 0, 5)))
h2 = h1_ocho_puzzle(MockNode((5, 2, 3, 0, 4, 8, 7, 6, 1)))
h3 = h1_ocho_puzzle(MockNode((1, 2, 3, 8, 0, 4, 7, 6, 5))) # Estado final
h4 = h1_ocho_puzzle(MockNode((1, 2, 3, 4, 0, 8, 7, 6, 5)))
h5 = h1_ocho_puzzle(MockNode((1, 2, 3, 8, 4, 0, 7, 6, 5))) # Hueco no cuenta
assert h1 == 4, h1
assert h2 == 4, h2
assert h3 == 0, h3 # Estado final
assert h4 == 2, h4
assert h5 == 1, h5 # 0 no debería influir

In [0]:
def dist(point1, point2):
  x1, y1 = point1
  x2, y2 = point2
  return abs(x2 - x1) + abs(y2 - y1)

assert dist((1, 1), (2, 2)) == 2

def h2_ocho_puzzle(nodo):
  estado = nodo.estado
  estado_final = (1, 2, 3, 8, 0, 4, 7, 6, 5)
  h = 0
  # implementar suma de la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final
  i1 = 0
  for c in estado_final:
    if c == 0:
      i1 += 1
      continue

    point2 = i1 % 3, int(i1 /3)
    i2 = estado.index(c)
    point1 = i2 % 3, int(i2 / 3)
    d = dist(point1, point2)
    if d != 0:
      h += d

    i1 += 1

  return h

# Tests
class MockNode:
  def __init__(self, estado): self.estado = estado
h1 = h2_ocho_puzzle(MockNode((2, 8, 3, 1, 6, 4, 7, 0, 5)))
h2 = h2_ocho_puzzle(MockNode((5, 2, 3, 0, 4, 8, 7, 6, 1)))
h3 = h2_ocho_puzzle(MockNode((1, 2, 3, 8, 0, 4, 7, 6, 5))) # Estado final
h4 = h2_ocho_puzzle(MockNode((3, 2, 1, 8, 0, 4, 7, 6, 5)))
h5 = h2_ocho_puzzle(MockNode((3, 2, 5, 8, 0, 4, 7, 1, 6))) # 2 + 2 + 3 + 1 = 8
assert h1 == 5, h1
assert h2 == 11, h2
assert h3 == 0, h3
assert h4 == 4, h4
assert h5 == 8, h5

---
## **Ejercicio 6**

Intentar resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el **problema del 8 puzzle** para los siguientes estados iniciales:

```
#           E1              E2              E3              E4
#           
#     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    
#     | 2 | 8 | 3 |   | 4 | 8 | 1 |   | 2 | 1 | 6 |   | 5 | 2 | 3 |
#     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
#     | 1 | 6 | 4 |   | 3 | H | 2 |   | 4 | H | 8 |   | H | 4 | 8 |
#     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
#     | 7 | H | 5 |   | 7 | 6 | 5 |   | 7 | 5 | 3 |   | 7 | 6 | 1 |
#     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
```


#### 1. Complete el bloque a continuación para generar cada uno de los estados inciales requeridos que luego usará en los experimentos

In [0]:
e1_8puzzle = (2, 8, 3, 1, 6, 4, 7, 0, 5)
e2_8puzzle = (4, 8, 1, 3, 0, 2, 7, 6, 5)
e3_8puzzle = (2, 1, 6, 4, 0, 8, 7, 5, 3)
e4_8puzzle = (5, 2, 3, 0, 4, 8, 7, 6, 1)

Se pide, en cada caso, hacerlo con la función resuelve_ocho_puzzle, para obtener, además de la solución, la longitud (el coste) de la solución obtenida, el número de nodos analizados y el tiempo requerido.

#### 2. **Complete el codigo** del siguiente bloque que ejecuta iterativamente todas las combinaciones  de búsquedas solicitadas en el experimento


In [0]:
# listas vacias que se llenaran con el avance del algoritmo
resultados_e1 = []
resultados_e2 = []
resultados_e3 = []
resultados_e4 = []
metodos = []

# Wrapper para definir cota a BúsquedaEnProfundidad en caso de que no encuentre la solución
def con_cota(algoritmo, cota):
    def BúsquedaEnProfundidadConCota():
      return algoritmo(cota) # Definido más arriba
    return BúsquedaEnProfundidadConCota

# Se cambia BúsquedaEnProfundidad por BúsquedaEnProfundidadAcotada
for busqueda_ciega in [BúsquedaEnAnchura, con_cota(BúsquedaEnProfundidadAcotada, 1000)]: 
    metodos.append(str(busqueda_ciega))
    resultados_e1.append(resuelve_ocho_puzzle(e1_8puzzle, busqueda_ciega))
    resultados_e2.append(resuelve_ocho_puzzle(e2_8puzzle,busqueda_ciega))
    resultados_e3.append(resuelve_ocho_puzzle(e3_8puzzle,busqueda_ciega))
    # resultados_e4.append(resuelve_ocho_puzzle(e4_8puzzle,busqueda_ciega))


for busqueda_informada in [BúsquedaPrimeroElMejor,BúsquedaAEstrella]:
    for heuristica in [h1_ocho_puzzle, h2_ocho_puzzle]:  
        metodos.append(str(busqueda_informada)+str(heuristica))
        resultados_e1.append(resuelve_ocho_puzzle(e1_8puzzle,busqueda_informada,heuristica))
        resultados_e2.append(resuelve_ocho_puzzle(e2_8puzzle,busqueda_informada,heuristica))
        resultados_e3.append(resuelve_ocho_puzzle(e3_8puzzle,busqueda_informada,heuristica))
        # resultados_e4.append(resuelve_ocho_puzzle(e4_8puzzle,busqueda_informada,heuristica))


[ BúsquedaEnAnchura ]
Solución: ['Mover hueco a arriba', 'Mover hueco a arriba', 'Mover hueco a izquierda', 'Mover hueco a abajo', 'Mover hueco a derecha']
Longitud de la solución: 5. Nodos analizados: 35, Tiempo de búsqueda: 0.003117084503173828
[ BúsquedaEnAnchura ]
Solución: ['Mover hueco a izquierda', 'Mover hueco a arriba', 'Mover hueco a derecha', 'Mover hueco a derecha', 'Mover hueco a abajo', 'Mover hueco a izquierda', 'Mover hueco a izquierda', 'Mover hueco a arriba', 'Mover hueco a derecha', 'Mover hueco a derecha', 'Mover hueco a abajo', 'Mover hueco a izquierda']
Longitud de la solución: 12. Nodos analizados: 2032, Tiempo de búsqueda: 1.0141735076904297
[ BúsquedaEnAnchura ]
Solución: ['Mover hueco a arriba', 'Mover hueco a izquierda', 'Mover hueco a abajo', 'Mover hueco a derecha', 'Mover hueco a derecha', 'Mover hueco a arriba', 'Mover hueco a izquierda', 'Mover hueco a izquierda', 'Mover hueco a abajo', 'Mover hueco a derecha', 'Mover hueco a derecha', 'Mover hueco a aba


El siguiente bloque de código construirá una tabla para reportar todos los resultados 

In [0]:
import pandas as pd

resultados_experimento = pd.DataFrame({
    'metodos' : metodos,
    'resultados_e1' : resultados_e1,
    'resultados_e2' : resultados_e2,
    'resultados_e3' : resultados_e3,
    # 'resultados_e4' : resultados_e4
})

resultados_experimento

,metodos,resultados_e1,resultados_e2,resultados_e3
0,<class '__main__.BúsquedaEnAnchura'>,"{'longitud': 5, 'analizados': 35, 'tiempo': 0....","{'longitud': 12, 'analizados': 2032, 'tiempo':...","{'longitud': 18, 'analizados': 22912, 'tiempo'..."
1,<function con_cota.<locals>.BúsquedaEnProfundi...,"{'longitud': 999, 'analizados': 1374, 'tiempo'...","{'longitud': 990, 'analizados': 16868, 'tiempo...","{'longitud': 998, 'analizados': 66014, 'tiempo..."
2,<class '__main__.BúsquedaPrimeroElMejor'><func...,"{'longitud': 5, 'analizados': 6, 'tiempo': 0.0...","{'longitud': 18, 'analizados': 354, 'tiempo': ...","{'longitud': 30, 'analizados': 237, 'tiempo': ..."
3,<class '__main__.BúsquedaPrimeroElMejor'><func...,"{'longitud': 5, 'analizados': 6, 'tiempo': 0.0...","{'longitud': 24, 'analizados': 202, 'tiempo': ...","{'longitud': 36, 'analizados': 227, 'tiempo': ..."
4,<class '__main__.BúsquedaAEstrella'><function ...,"{'longitud': 5, 'analizados': 7, 'tiempo': 0.0...","{'longitud': 12, 'analizados': 119, 'tiempo': ...","{'longitud': 18, 'analizados': 1747, 'tiempo':..."
5,<class '__main__.BúsquedaAEstrella'><function ...,"{'longitud': 5, 'analizados': 6, 'tiempo': 0.0...","{'longitud': 12, 'analizados': 29, 'tiempo': 0...","{'longitud': 18, 'analizados': 227, 'tiempo': ..."


Analizar los resultados de la  tabla y justificarlos con las distintas propiedades teóricas estudiadas.


# Análisis

La **Búsqueda en anchura** obtiene la solución óptima ya que estamos considerando el costo de acción como uniforme. Sin embargo, es complejo espacialmente.

Fue necesario colocarle a **Búsqueda en profundidad** una cota, en caso contrario nunca se encontraría el camino solución. O tal vez, podría llegar a demorar horas. Mediante la cota se logra que llegado a determinado límite de profundidad, se expandan otras ramas (se vuelve por la rama actual para explorar otras). Asegurando así que en algún momento encontrará la solución y no explorará infinitamente la misma rama. Así, **BúsquedaEnProfunidadAcotada** permite la completitud. Pero no el óptimo.

Las búsquedas informadas tienen menor complejidad espacial y temporal que las ciegas, ya que mediante la heurística se guía la exploración para encontrar el camino solución. Sin embargo, **Búsqueda primero el mejor** o Voraz demora más tiempo y es más compleja espacialmente que una búsqueda **A\***.

Para este tipo de problema, la **Búsqueda A\*** resultó la más apropiada en términos de complejidad temporal y espacial. Considerando tanto el costo de camino hasta el nodo actual y una heurística **admisible**, guía la exploración para encontrar el camino solución óptimo, y también, lo hace de forma óptima (realizando el menor recorrido posible).

Todos los algoritmos encontraron caminos solución similares, variando en, relativamente, algunos pasos, excepto al utilizar la búsqueda en profunidad, donde esto se disparó a caminos soluciones cuya longitud es cercana a 1000 (debido a la cota). Además, se exploraron una gran cantidad de nodos en comparación a los demás algoritmos.